In [1]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random
import math

# List of robot available movements
MOVES = [(0, -1), (0, 1), (1, 0), (-1,0)]

In [9]:
class CityModel(ap.Model):

    def setup(self):
        M = self.p.M
        nCars = int(self.p.nCars)
        street = int(M ** 2)
        
        # Defining agents 
        self.car = ap.AgentList(self,nCars)

        self.street = ap.AgentList(self, street)
        self.street.agent_type = 0

        # Defining grid size
        self.space = ap.Grid(self, [M] * 2, track_empty=True)

        # Add agents to grid 
        self.space.add_agents(self.street, random=True, empty=True)
    
        street1 = self.space.agents[0:M, 28:33]
        street2 = self.space.agents[28:33,0:M]  
        street1.agent_type = 1
        street2.agent_type = 1
        
        positions =self.getStreetPositions()
        self.space.add_agents(self.car, positions)
        
        self.car.agent_type = 2

        #Agent_type: {0:space, 1:street, 2: car}

    def getStreetPositions(self):
        positions= []
        for index in range(len(self.car)):
            if index % 2 == 0:
                positions.append((30,0+index))
            else:
                positions.append((0+index,30))
        return positions


    def step(self):
        cars = self.car
        for i, car in enumerate(cars):
            if i % 2  == 0:
                self.space.move_by(car, (0,1))
            else:
                self.space.move_by(car,(1,0))



    def end(self):
        # Document a measure at the end of the simulation
        self.report('Percentage of cleaned tiles',2)
        print(self.report['Percentage of cleaned tiles'])

In [5]:
# Define parameters

parameters = {
    'nCars': 10, # Number of cleaning robot agents
    'M': 60, # city size
    'steps': 50 # Number of simulation steps
}

In [12]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.space.attr_grid('agent_type')
    color_dict = {0:'#000', 1:'#c1c1c1', 2:'#4cbff5', 'None' :'#FFF'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a Robot cleaning a Room\n"
                 f"Time-step: {model.t}",
                 color="white") 

fig, ax = plt.subplots()
model = CityModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=30))